<a id="top"></a>

<div class="list-group" id="list-tab" role="tablist">
<h3 class="list-group-item list-group-item-action active" data-toggle="list" style='background:blue; border:0' role="tab" aria-controls="home" color=green><center>The Social Dilemma, a documentary-drama hybrid explores the dangerous human impact of social networking as the tech experts sound the alarm on the dangerous human impact of social networking.</center></h3>

![](https://www.thenews.com.pk/assets/uploads/tns/2019-07-07/568081_5906029_tns.jpg)

## This notebook analyses the tweets with the trending #TheSocialDilemma hashtag made by the users of twitter!

<a id="top"></a>

<div class="list-group" id="list-tab" role="tablist">
<h3 class="list-group-item list-group-item-action active" data-toggle="list" style='background:black; border:0' role="tab" aria-controls="home" color=green><center>Quick navigation</center></h3>

* [1. Required Libraries](#1)
* [2. Dataset Quick Overview](#2)
* [3. Tweets EDA](#3)
* [4. Tweets text analysis](#4)   

    Kindly, Upvote the notebook!

<a id="1"></a>
<h2 style='background:green; border:0; color:white'><center>Required Libraries</center><h2><a id="1"></a>


In [ ]:
import numpy as np 
import pandas as pd 
import os
import itertools

#plots
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.colors import n_colors
from plotly.subplots import make_subplots

from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from sklearn.feature_extraction.text import CountVectorizer

from PIL import Image
from nltk.corpus import stopwords
stop=set(stopwords.words('english'))
from nltk.util import ngrams


import re
from collections import Counter

import nltk
from nltk.corpus import stopwords

import requests
import json

import seaborn as sns
sns.set(rc={'figure.figsize':(11.7,8.27)})


<a id="2"></a>
<h2 style='background:green; border:0; color:white'><center>Dataset Quick Overview</center><h2>

In [ ]:
social=pd.read_csv('../input/the-social-dilemma-tweets/TheSocialDilemma.csv')
social.info()

### The dataset consists of 18,252 tweets with 14 columns!

## Let's visualize some missing values!

In [ ]:
import missingno as mno
mno.matrix(social)

In [ ]:
missed = pd.DataFrame()
missed['column'] = social.columns

missed['percent'] = [round(100* social[col].isnull().sum() / len(social), 2) for col in social.columns]
missed = missed.sort_values('percent',ascending=False)
missed = missed[missed['percent']>0]

fig = sns.barplot(
    x=missed['percent'], 
    y=missed["column"], 
    orientation='horizontal'
).set_title('Missed values percent for every column')

<a id="3"></a>
<h2 style='background:green; border:0; color:white'><center>Tweets EDA</center><h2>

### Let's visualize the sentiment of the tweets!

In [ ]:
ds = social['Sentiment'].value_counts().reset_index()
ds.columns = ['Sentiment', 'Count']
ds = ds.sort_values(['Count'],ascending=False)
#social = pd.merge(social, ds, on='user_name')

fig = sns.barplot( 
    x=ds["Sentiment"], 
    y=ds["Count"], 
    orientation='vertical'
).set_title('Sentiment of the tweets') 

* Most of the tweets are positive in nature, which denotes a wide appreciation of the documentary among the users!

## Lets Visualize the top 20 users by number of tweets


In [ ]:
ds = social['user_name'].value_counts().reset_index()
ds.columns = ['user_name', 'tweets_count']
ds = ds.sort_values(['tweets_count'],ascending=False)
social = pd.merge(social, ds, on='user_name')

fig = sns.barplot( 
    x=ds.head(20)["tweets_count"], 
    y=ds.head(20)["user_name"], 
    orientation='horizontal'
).set_title('Top 20 users by number of tweets') 



### The user "OurPact" has made the highest number of tweets! No let's look into OurPact's tweets alone!

In [ ]:
social[social['user_name']=='OurPact'][['text','Sentiment']]

#### Let's check the sentiment of the tweets made by the user name OurPact

In [ ]:
social[social['user_name']=='OurPact']['Sentiment'].value_counts()

The user name OurPact has created nearly 218 tweets with 213(majority) tweets in positive sentiment! Looks like Ourpact loved the documentary!

### User created year by year

In [ ]:
social['user_created'] = pd.to_datetime(social['user_created'])
social['year_created'] = social['user_created'].dt.year
data = social.drop_duplicates(subset='user_name', keep="first")
data = data[data['year_created']>1970]
data = data['year_created'].value_counts().reset_index()
data.columns = ['year', 'number']

fig = sns.barplot( 
    x=data["year"], 
    y=data["number"], 
    orientation='vertical'
    #title='', 
).set_title('User created year by year')

* 2009 has the highest number of users followed by the year 2011, who tweeted about the social dilemma documentary
* The amount of users who joined in 2008 tweeted very less about the social dilemma documentary comapred to the other users who joined in the later years!


## Top 20 Users location based on the number of tweets

In [ ]:
ds = social['user_location'].value_counts().reset_index()
ds.columns = ['user_location', 'count']
ds = ds[ds['user_location']!='NA']
ds = ds.sort_values(['count'],ascending=False)

fig = sns.barplot(
    
    x=ds.head(20)["count"], 
    y=ds.head(20)["user_location"], 
    orientation='horizontal'
).set_title('Top 20 user locations by number of tweets')

#### India holds the most number of tweets by location followed by San diego and Los angeles, California!

## Visualizing the number of tweets per location!!

In [ ]:
from plotly.offline import init_notebook_mode, iplot
def pie_count(data, field, percent_limit, title):
    
    data[field] = data[field].fillna('NA')
    data = data[field].value_counts().to_frame()

    total = data[field].sum()
    data['percentage'] = 100 * data[field]/total    

    percent_limit = percent_limit
    otherdata = data[data['percentage'] < percent_limit] 
    others = otherdata['percentage'].sum()  
    maindata = data[data['percentage'] >= percent_limit]

    data = maindata
    other_label = "Others(<" + str(percent_limit) + "% each)"
    data.loc[other_label] = pd.Series({field:otherdata[field].sum()}) 
    
    labels = data.index.tolist()   
    datavals = data[field].tolist()
    
    trace=go.Pie(labels=labels,values=datavals)
    
    layout = go.Layout(
        title = title,
        height=600,
        width=600
        )
    
    fig = go.Figure(data=[trace], layout=layout)
    iplot(fig)
    
pie_count(social, 'user_location', 0.5, 'Number of tweets per location')

## Top 10 user sources by number of tweets

In [ ]:
ds = social['source'].value_counts().reset_index()
ds.columns = ['source', 'count']
ds = ds.sort_values(['count'],ascending=False)

fig = sns.barplot(
    x=ds.head(10)["count"], 
    y=ds.head(10)["source"], 
    orientation='horizontal', 
).set_title('Top 10 user sources by number of tweets')

### The most twitter tweets are made through the iphone! Followed by the android and very less people prefer web app comapred to iphone or android!

## Total number of tweets for users and number of hashtags in every tweet

In [ ]:
social['hashtags'] = social['hashtags'].fillna('[]')
social['hashtags_count'] = social['hashtags'].apply(lambda x: len(x.split(',')))
social.loc[social['hashtags'] == '[]', 'hashtags_count'] = 0
fig = sns.scatterplot( 
    x=social['hashtags_count'], 
    y=social['tweets_count']
).set_title('Total number of tweets for users and number of hashtags in every tweet')

## Number of hashtags used in each tweet

In [ ]:
ds = social['hashtags_count'].value_counts().reset_index()
ds.columns = ['hashtags_count', 'count']
ds = ds.sort_values(['count'],ascending=False)
ds['hashtags_count'] = ds['hashtags_count'].astype(str) + ' tags'
fig = sns.barplot( 
    x=ds["count"], 
    y=ds["hashtags_count"], 
    orientation='horizontal'
).set_title('Distribution of number of hashtags in tweets')

* Most users use 1 hastag followed by 2 hashtag, where certain population uses no hastag while tweeting
* Very less amount of people use more than 2 hashtags in their post 

In [ ]:
social['date'] = pd.to_datetime(social['date']) 
df = social.sort_values(['date'])
df['day'] = df['date'].astype(str).str.split(' ', expand=True)[0]
df['time'] = df['date'].astype(str).str.split(' ', expand=True)[1]
df.head()

ds = df.groupby(['day', 'user_name'])['hashtags_count'].count().reset_index()
ds = ds.groupby(['day'])['user_name'].count().reset_index()
ds.columns = ['day', 'number_of_users']
ds['day'] = ds['day'].astype(str)
fig = sns.barplot( 
    x=ds['day'], 
    y=ds["number_of_users"], 
    orientation='vertical',
    #title='Number of unique users per day', 
    #width=800, 
    #height=800
).set_title('Number of unique users per day')
#fig.show()
plt.xticks(rotation=90)

## Tweets distribution over days present in dataset

In [ ]:
ds = df['day'].value_counts().reset_index()
ds.columns = ['day', 'count']
ds = ds.sort_values('count',ascending=False)
ds['day'] = ds['day'].astype(str)
fig = sns.barplot( 
    x=ds['count'], 
    y=ds["day"], 
    orientation="horizontal",
).set_title('Tweets distribution over days present in dataset')

## Tweets per day

In [ ]:
social['tweet_date']=pd.to_datetime(social['date']).dt.date
tweet_date=social['tweet_date'].value_counts().to_frame().reset_index().rename(columns={'index':'date','tweet_date':'count'})
tweet_date['date']=pd.to_datetime(tweet_date['date'])
tweet_date=tweet_date.sort_values('date',ascending=False)

In [ ]:
fig=go.Figure(go.Scatter(x=tweet_date['date'],
                                y=tweet_date['count'],
                               mode='markers+lines',
                               name="Submissions",
                               marker_color='dodgerblue'))

fig.update_layout(
    title_text='Tweets per Day : ({} - {})'.format(social['tweet_date'].sort_values()[0].strftime("%d/%m/%Y"),
                                                       social['tweet_date'].sort_values().iloc[-1].strftime("%d/%m/%Y")),template="plotly_dark",
    title_x=0.5)

fig.show()

### The interesting factor is that the documentary was intially released on jan, 2020! Once the documentary was released in one of the major OTT platform(Netflix) on sept 9th, 2020! It started creating a quite a buzz in social media platforms like twitter, where people started to voice out their concerns on the impact of social media! 

### This is the reason where you identiy an steady increase in the number of tweets after sept 9th, 2020 but after a weeks time, the hype among people gradually decreased!

## Tweet distribution - hourly

In [ ]:
social['hour'] = social['date'].dt.hour
ds = social['hour'].value_counts().reset_index()
ds.columns = ['hour', 'count']
ds['hour'] = 'Hour ' + ds['hour'].astype(str)
fig = sns.barplot( 
    x=ds["hour"], 
    y=ds["count"], 
    orientation='vertical', 
).set_title('Tweets distribution over hours')
plt.xticks(rotation='vertical')


### Most number of tweets were made during 6 and 8 PM in the evenings and majorly most of the tweets were made during the evenings!

### Top 10 hastags used in the tweet!

In [ ]:
def split_hashtags(x): 
    return str(x).replace('[', '').replace(']', '').split(',')

tweets_df = social.copy()
tweets_df['hashtag'] = tweets_df['hashtags'].apply(lambda row : split_hashtags(row))
tweets_df = tweets_df.explode('hashtag')
tweets_df['hashtag'] = tweets_df['hashtag'].astype(str).str.lower().str.replace("'", '').str.replace(" ", '')
tweets_df.loc[tweets_df['hashtag']=='', 'hashtag'] = 'NO HASHTAG'
#tweets_df

In [ ]:
ds = tweets_df['hashtag'].value_counts().reset_index()
ds.columns = ['hashtag', 'count']
ds = ds.sort_values(['count'],ascending=False)
fig = sns.barplot(
    x=ds.head(10)["count"], 
    y=ds.head(10)['hashtag'], 
    orientation='horizontal', 
).set_title('Top 10 hashtags')

<a id="4"></a>
<h2 style='background:green; border:0; color:white'><center>Tweets text analysis</center><h2>

## Prevalent words in tweets 

In [ ]:
def build_wordcloud(df, title):
    wordcloud = WordCloud(
        background_color='black',colormap="Oranges", 
        stopwords=set(STOPWORDS), 
        max_words=50, 
        max_font_size=40, 
        random_state=666
    ).generate(str(df))

    fig = plt.figure(1, figsize=(14,14))
    plt.axis('off')
    fig.suptitle(title, fontsize=16)
    fig.subplots_adjust(top=2.3)

    plt.imshow(wordcloud)
    plt.show()

In [ ]:
build_wordcloud(social['text'], 'Prevalent words in tweets for all dataset')

## Prevalent words in tweets from India

In [ ]:
india_df = social.loc[social.user_location=="India"]
build_wordcloud(india_df['text'], title = 'Prevalent words in tweets from India')

## Refining the text (Important step)

In [ ]:
def remove_tag(string):
    text=re.sub('<.*?>','',string)
    return text
def remove_mention(text):
    line=re.sub(r'@\w+','',text)
    return line
def remove_hash(text):
    line=re.sub(r'#\w+','',text)
    return line

def remove_newline(string):
    text=re.sub('\n','',string)
    return text
def remove_url(string): 
    text = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+','',string)
    return text
def remove_number(text):
    line=re.sub(r'[0-9]+','',text)
    return line
def remove_punct(text):
    line = re.sub(r'[!"\$%&\'()*+,\-.\/:;=#@?\[\\\]^_`{|}~]*','',text)
    return line
def text_strip(string):
    line=re.sub('\s{2,}', ' ', string.strip())
    return line
def remove_thi_amp_ha_words(string):
    line=re.sub(r'\bamp\b|\bthi\b|\bha\b',' ',string)
    return line

In [ ]:
social['refine_text']=social['text'].str.lower()
social['refine_text']=social['refine_text'].apply(lambda x:remove_tag(str(x)))
social['refine_text']=social['refine_text'].apply(lambda x:remove_mention(str(x)))
social['refine_text']=social['refine_text'].apply(lambda x:remove_hash(str(x)))
social['refine_text']=social['refine_text'].apply(lambda x:remove_newline(x))
social['refine_text']=social['refine_text'].apply(lambda x:remove_url(x))
social['refine_text']=social['refine_text'].apply(lambda x:remove_number(x))
social['refine_text']=social['refine_text'].apply(lambda x:remove_punct(x))
social['refine_text']=social['refine_text'].apply(lambda x:remove_thi_amp_ha_words(x))
social['refine_text']=social['refine_text'].apply(lambda x:text_strip(x))

social['text_length']=social['refine_text'].str.split().map(lambda x: len(x))

## The average length for a IPL2020 Tweet using violin plot

In [ ]:
fig = go.Figure(data=go.Violin(y=social['text_length'], box_visible=True, line_color='black',
                               meanline_visible=True, fillcolor='royalblue', opacity=0.6,
                               x0='Tweet Text Length'))

fig.update_layout(yaxis_zeroline=False,title="Distribution of Text length",template='ggplot2')
fig.show()

* Average length of the ipl2020 tweet: 13.06
* Median lenght of the ipl 2020 tweet:15
* Interquartile lie between : 10 and 18
* Min: 0
* Max: 27

## N-GRAM

## Listing below the top N-gram sequential words used in IPL2020 tweets

In [ ]:
def ngram_df(corpus,nrange,n=None):
    vec = CountVectorizer(stop_words = 'english',ngram_range=nrange).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    total_list=words_freq[:n]
    df=pd.DataFrame(total_list,columns=['text','count'])
    return df
unigram_df=ngram_df(social['refine_text'],(1,1),20)
bigram_df=ngram_df(social['refine_text'],(2,2),20)
trigram_df=ngram_df(social['refine_text'],(3,3),20)

In [ ]:
fig = make_subplots(
    rows=3, cols=1,subplot_titles=("Unigram","Bigram",'Trigram'),
    specs=[[{"type": "scatter"}],
           [{"type": "scatter"}],
           [{"type": "scatter"}]
          ])

fig.add_trace(go.Bar(
    y=unigram_df['text'][::-1],
    x=unigram_df['count'][::-1],
    marker={'color': "blue"},  
    text=unigram_df['count'],
    textposition = "outside",
    orientation="h",
    name="Months",
),row=1,col=1)

fig.add_trace(go.Bar(
    y=bigram_df['text'][::-1],
    x=bigram_df['count'][::-1],
    marker={'color': "blue"},  
    text=bigram_df['count'],
     name="Days",
    textposition = "outside",
    orientation="h",
),row=2,col=1)

fig.add_trace(go.Bar(
    y=trigram_df['text'][::-1],
    x=trigram_df['count'][::-1],
    marker={'color': "blue"},  
    text=trigram_df['count'],
     name="Days",
    orientation="h",
    textposition = "outside",
),row=3,col=1)

fig.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
fig.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
fig.update_layout(title_text='Top N Grams',xaxis_title=" ",yaxis_title=" ",
                  showlegend=False,title_x=0.5,height=1200,template="plotly_dark")
fig.show()

* social, watch, and media are the most used unigrams
* social media, social dilemma, just watched are the most used bigrams in the social dilemma tweets
* social dilemma neflix, wach social dilemma, keepin kids safe, kids safe online are the most used trigrams!


<h2 style='background:green; border:0; color:white'><center> PART 2: Twitter sentiment classification models and techniques! Coming soon! Stay tuned! </center><h2>



[Dataset Link with 10,000 and more tweets with #TheSocialDilemma hashtag](https://www.kaggle.com/kaushiksuresh147/the-social-dilemma-tweets)

<h2 style='background:green; border:0; color:white'><center>Kindly upvote the notebook and the dataset!</center><h2>